<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/%E4%B8%89%E7%B8%BD%E5%8C%97%E6%8A%95%E9%96%80%E8%A8%BA%E5%8A%A9%E6%89%8B3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##三總北投門診助手 3.0  2021/8/22

In [1]:
!pip install flask_ngrok
!pip install line-bot-sdk
!pip install bs4
!pip install requests

     |████████████████████████████████| 72 kB 709 kB/s 


In [2]:
import OPDAid3_SN
OPDAid3_SN.myLineBotApi
OPDAid3_SN.myWebhookHandler

'17a979c412f4bb0adb0014f3fcc76708'

In [16]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from datetime import date
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import (MessageEvent, TextMessage, TextSendMessage)
from bs4 import BeautifulSoup
import requests, json
import OPDAid3_SN
today=date.today()
app=Flask(__name__)

line_bot_api=LineBotApi(OPDAid3_SN.myLineBotApi)
handler=WebhookHandler(OPDAid3_SN.myWebhookHandler)

run_with_ngrok(app)

@app.route('/')
def index():
  return '三總北投門診助手3.0 '+str(today)

@app.route('/callback',methods=['POST'])
def callback():
  signature=request.headers['X-Line-Signature']
  body=request.get_data(as_text=True)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

'''
def urlRequest(target_url):
  from bs4 import BeautifulSoup
  import requests
  responses=requests.get(target_url)
  soup=BeautifulSoup(responses.text,"html.parser")
  myDivs=soup.find_all('div',{'class':'column column-block lg-txt'})
  myResult=''
  for myDiv in myDivs:
    myLink=myDiv.find('a')
    myResult += str(myLink.contents[0])
    myResult += '\n'
    myResult += f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink.get('href')}"
    myResult += '\n\n'
  return(myResult)
'''

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
  from bs4 import BeautifulSoup
  import requests
  msg=event.message.text
  if msg=='醫師查詢':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/Doclist/194/30000/25075'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    myDivs=soup.find_all('div',{'class':'column column-block lg-txt'})
    myResult=''
    for myDiv in myDivs:
      myLink=myDiv.find('a')
      myResult += str(myLink.contents[0])
      myResult += '\n'
      myResult += f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink.get('href')}"
      myResult += '\n\n'   
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))
  if msg=='門診時間':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/28387'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    門診時刻表=soup.select_one('div.systems-information')
    每月時刻表=門診時刻表.find_all('a')
    myResult=''
    for tag in 每月時刻表:
      myLink=str(tag.get('href'))
      myLink=f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink}"
      myResult +=tag.contents[0]
      myResult +='\n'
      myResult += myLink
      myResult +='\n'
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='掛號查詢':  
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msg+'維修中'))

  if msg=='費用申請':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/16048'
    response=requests.get(target_url)
    soup=BeautifulSoup(response.text, 'html.parser')
    myDiv=soup.select_one('div.systems-information')
    收費標準s=myDiv.find_all('a')
    myResult=''
    for i, tag in enumerate(收費標準s):
      myResult+='\n {}.{}。詳情如下: \n {}'.format(i,tag.contents[0],tag.get('href'))
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='重大公告':
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msg+'維修中'))

  else:
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msg))

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://50ff-34-125-95-239.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Aug/2021 03:23:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Aug/2021 03:23:14] "GET /favicon.ico HTTP/1.1" 404 -
[2021-08-22 03:23:39,585] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.e